In [1]:
import pandas as pd
import requests
import json
import time
from datetime import datetime

from api_key import weather_key2

In [2]:
#Insert dataframe of csv data here
data=pd.read_csv('Resources/aviation_renamed.csv')
data.head(10)

,Unnamed: 0,Date,Country,City_State,Make,Model,Injury_Severity,Aircraft_Damage,Latitude,Longitude
0,0,12/31/2019,United States,"Elk, CA",Cessna,175,Non-Fatal,Substantial,39.128611,-123.715833
1,1,12/31/2019,United States,"OLATHE, KS",Mooney,M20S,Fatal(2),Destroyed,38.846111,-94.736111
2,2,12/31/2019,United States,"Fairbanks, AK",Cessna,170,Non-Fatal,Substantial,64.666945,-148.133334
3,5,12/28/2019,United States,"LAFAYETTE, LA",Piper,PA 31T,Fatal(5),Destroyed,30.176111,-92.007500
4,6,12/25/2019,United States,"Headland, AL",Bell,407,Fatal(1),Substantial,31.364167,-85.312500
5,9,12/22/2019,United States,"Lake Havasu, AZ",Cosmos,Phase II,Non-Fatal,Substantial,34.440000,-114.345555
6,10,12/21/2019,United States,"Evansville, IN",Piper,PA28,Fatal(1),Destroyed,38.095833,-87.540555
7,12,12/19/2019,United States,"Beeville, TX",Robinson,R22,Non-Fatal,Substantial,28.367500,-97.796389
8,13,12/18/2019,United States,"Caldwell, ID",Piper,PA22,Non-Fatal,Substantial,43.643889,-116.636944
9,14,12/18/2019,United States,"Harrison, OH",Cessna,172,Non-Fatal,Substantial,39.259167,-84.774445


In [3]:
#Fake database

#data=pd.DataFrame({'City Name': ['a', 'b', '', '',''], 
#                        'Lat': ['39.128611', '38.846111', '64.666945', '32.365556', '30.176111'], 
#                        'Long': ['-123.715833', '-94.736111', '-148.133334', '-97.645', '-92.0075'],
#                        'Date':['2019/12/31', '2019/12/31', '2019/12/31', '2019/12/30', '2019/12/28'],
#                        'Max Temp': ['', '','','',''], 
#                        'Cloudiness': ['', '','','',''],
#                        'Humidity': ['', '','','',''], 
#                        'Wind Speed':['', '','','','']
#                        })

In [4]:
weather_data_df=pd.DataFrame({'Date':[],
                              'Latitude':[],
                              'Longitude':[],
                              'Incident Index':[],
                              'Temperature':[],
                              'Max Temperature':[],
                              'Wind Speed': [],
                              'Visibility':[],
                              'Precipitation':[],
                              'Humidity':[],
                              'Conditions':[],
                              'Cloudcover':[]})
weather_data_df

,Date,Latitude,Longitude,Incident Index,Temperature,Max Temperature,Wind Speed,Visibility,Precipitation,Humidity,Conditions,Cloudcover


In [5]:
#https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations=Herndon,VA,20170&aggregateHours=24&unitGroup=us&shortColumnNames=false&contentType=json&key=YOURAPIKEY

base_url='https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?locations='

for i in range(len(data)):
    date=data.Date[i].split('/')
    date=date[2]+'-'+date[0]+'-'+date[1]

    latlong=f'{data.Latitude[i]},{data.Longitude[i]}'
    loc='&aggregateHours=24&unitGroup=us&shortColumnNames=false&contentType=json'
    dt=f'&startDateTime={date}T00%3A00%3A00'
    key=f'&key={weather_key2}'
    url=base_url+latlong+loc+dt+key
    try:
        weather_response = requests.get(url)
        weather_json = weather_response.json()
        weather_json['locations']
        weather_data_single=pd.DataFrame({'Date':[weather_json['locations'][latlong]['values'][0]['datetimeStr']],
                                          'Latitude':[data.Latitude[i]],
                                          'Longitude':[data.Longitude[i]],
                                          'Incident Index':[data['Unnamed: 0'][i]],
                                          'Temperature':[weather_json['locations'][latlong]['values'][0]['temp']],
                                          'Max Temperature':[weather_json['locations'][latlong]['values'][0]['maxt']],
                                          'Wind Speed':[weather_json['locations'][latlong]['values'][0]['wspd']],
                                          'Visibility':[weather_json['locations'][latlong]['values'][0]['visibility']],
                                          'Precipitation':[weather_json['locations'][latlong]['values'][0]['precip']],
                                          'Humidity':[weather_json['locations'][latlong]['values'][0]['humidity']],
                                          'Conditions':[weather_json['locations'][latlong]['values'][0]['conditions']],
                                          'Cloudcover':[weather_json['locations'][latlong]['values'][0]['cloudcover']]
        })
    except:
        print("No value found")
        print(date) #Add and print id of not found
    weather_data_df=weather_data_df.append(weather_data_single)
    time.sleep(1)


weather_data_df

und
2004-4-17
No value found
2004-4-17
No value found
2004-4-16
No value found
2004-4-16
No value found
2004-4-15
No value found
2004-4-13
No value found
2004-4-12
No value found
2004-4-12
No value found
2004-4-10
No value found
2004-4-10
No value found
2004-4-9
No value found
2004-4-9
No value found
2004-4-7
No value found
2004-4-5
No value found
2004-4-4
No value found
2004-4-4
No value found
2004-3-31
No value found
2004-3-31
No value found
2004-3-29
No value found
2004-3-28
No value found
2004-3-26
No value found
2004-3-21
No value found
2004-3-21
No value found
2004-3-18
No value found
2004-3-17
No value found
2004-3-13
No value found
2004-3-13
No value found
2004-3-12
No value found
2004-3-9
No value found
2004-3-9
No value found
2004-3-7
No value found
2004-3-7
No value found
2004-3-6
No value found
2004-3-6
No value found
2004-3-6
No value found
2004-2-28
No value found
2004-2-28
No value found
2004-2-23
No value found
2004-2-22
No value found
2004-2-21
No value found
2004-2-21

,Date,Latitude,Longitude,Incident Index,Temperature,Max Temperature,Wind Speed,Visibility,Precipitation,Humidity,Conditions,Cloudcover
0,2019-12-31T00:00:00-08:00,39.128611,-123.715833,0.0,48.8,58.2,5.2,7.6,0.00,71.77,Clear,14.1
0,2019-12-31T00:00:00-06:00,38.846111,-94.736111,1.0,33.7,42.2,13.9,9.9,0.00,59.91,Clear,0
0,2019-12-31T00:00:00-09:00,64.666945,-148.133334,2.0,-3.0,-0.3,4.5,9.3,0.00,76.68,Partially cloudy,74.6
0,2019-12-28T00:00:00-06:00,30.176111,-92.007500,5.0,68.9,76.0,12.8,5.5,0.09,92.13,"Rain, Overcast",89.9
0,2019-12-25T00:00:00-06:00,31.364167,-85.312500,6.0,60.2,72.0,10.3,9.9,0.00,66.3,Clear,0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,1983-09-09T00:00:00-06:00,48.120000,-113.887500,78052.0,48.6,57.8,16.1,26.6,0.00,70.39,Overcast,93
0,1983-01-08T00:00:00-08:00,46.041111,-120.849722,80600.0,45.1,51.2,27.7,19.5,0.31,68.96,"Snow, Partially cloudy",65.6
0,1982-03-16T00:00:00-06:00,30.757778,-88.355555,83668.0,74.6,80.9,20.8,8.7,0.00,80.43,Overcast,96.5
0,1979-09-17T00:00:00-04:00,42.445277,-70.758333,84256.0,65.5,78.2,15,13.3,0.00,69.99,Clear,2.6


In [6]:
weather_data_df=weather_data_df.reset_index(drop=True)
weather_data_df

,Date,Latitude,Longitude,Incident Index,Temperature,Max Temperature,Wind Speed,Visibility,Precipitation,Humidity,Conditions,Cloudcover
0,2019-12-31T00:00:00-08:00,39.128611,-123.715833,0.0,48.8,58.2,5.2,7.6,0.00,71.77,Clear,14.1
1,2019-12-31T00:00:00-06:00,38.846111,-94.736111,1.0,33.7,42.2,13.9,9.9,0.00,59.91,Clear,0
2,2019-12-31T00:00:00-09:00,64.666945,-148.133334,2.0,-3.0,-0.3,4.5,9.3,0.00,76.68,Partially cloudy,74.6
3,2019-12-28T00:00:00-06:00,30.176111,-92.007500,5.0,68.9,76.0,12.8,5.5,0.09,92.13,"Rain, Overcast",89.9
4,2019-12-25T00:00:00-06:00,31.364167,-85.312500,6.0,60.2,72.0,10.3,9.9,0.00,66.3,Clear,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24008,1983-09-09T00:00:00-06:00,48.120000,-113.887500,78052.0,48.6,57.8,16.1,26.6,0.00,70.39,Overcast,93
24009,1983-01-08T00:00:00-08:00,46.041111,-120.849722,80600.0,45.1,51.2,27.7,19.5,0.31,68.96,"Snow, Partially cloudy",65.6
24010,1982-03-16T00:00:00-06:00,30.757778,-88.355555,83668.0,74.6,80.9,20.8,8.7,0.00,80.43,Overcast,96.5
24011,1979-09-17T00:00:00-04:00,42.445277,-70.758333,84256.0,65.5,78.2,15,13.3,0.00,69.99,Clear,2.6


In [7]:
weather_data_df.to_csv('Resources/weather_data_raw.csv')